# VIDEO DETECT

In [7]:
import cv2
import numpy as np
import random
import math

# Load Yolo
net = cv2.dnn.readNet("yolo/project-obj.cfg","yolo/project-obj_best.weights")#path of cfg and weights file
#net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)

classes = []
with open("yolo/obj.names",'r') as f:   # path of classes.txt file OR obj.names file
    classes = f.read().splitlines()

writer = None

cap = cv2.VideoCapture("phase1.mp4") #path of video ,if 0 it is set for webcam


while True:

    _, frame = cap.read()
    height, width, _ = frame.shape

    blob = cv2.dnn.blobFromImage(frame, 1/255, (416,416),(0,0,0), True, crop = False)

    net.setInput(blob)

    
    output_layers_names = net.getUnconnectedOutLayersNames()
    layerOutputs = net.forward(output_layers_names)
    

    boxes = []
    confidences = []
    class_ids = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                x = int(center_x - w/2)
                y = int(center_y - h/2)

                boxes.append([x, y, w, h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    FONT = cv2.FONT_HERSHEY_PLAIN
    COLORS = np.random.uniform(0, 255, size =(len(boxes),3))

    if len(indexes) > 0:
        for i in indexes.flatten():

            (x,y) = (boxes[i][0], boxes[i][1])
            (w,h) = (boxes[i][2], boxes[i][3])
            centeriX = boxes[i][0] + (boxes[i][2] // 2)
            centeriY = boxes[i][1] + (boxes[i][3] // 2)

            #color = [int(c) for c in COLORS[classIDs[i]]]
            text = "{}: {:.2f}".format(COLORS[class_ids[i]], confidences[i])

            indexes_copy = list(indexes.flatten())
            indexes_copy.remove(i)

            for j in np.array(indexes_copy):
                centerjX = boxes[j][0] + (boxes[j][2] // 2)
                centerjY = boxes[j][1] + (boxes[j][3] // 2) 

                distance = math.sqrt(math.pow(centerjX - centeriX , 2) + math.pow(centerjY - centeriY , 2))

                if distance <= 200:
                    cv2.line(frame, (boxes[i][0] + (boxes[i][2] // 2),boxes[i][1]  + (boxes[i][3] // 2)), 
                     (boxes[j][0] + (boxes[j][2] // 2), boxes[j][1] + (boxes[j][3] // 2)), (0, 0, 255), 2)
                    # unSafe.append([centerjX,centerjY])
                    # unSafe.append([centeriX,centeriY])

            # if centeriX in chain(*unSafe) and centeriY in chain(*unSafe):
                # print(f"The distance between the object {distance}")
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255,255,0), 2)
            cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
    
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"XVID")
        writer = cv2.VideoWriter("distanceOutput.avi", fourcc, 30, (frame.shape[1], frame.shape[0]), True)
    writer.write(frame)
    
   
        #croppedImage = img[y:y+h,x:x+w]
        #cv2.imwrite("image.png",croppedImage)     
    cv2.imshow("Video",frame)
    if cv2.waitKey(1) > 0:
        break
writer.release()
cap.release()
cv2.destroyAllWindows()